# Actividad NLP 3

Mariana Castro Payns

Marcela Ibarra Mora

María de los Ángeles Arista Huerta

Eduardo Rodríguez Gil

Manolo Ramírez Pintor

## English-Spanish translator 

https://www.manythings.org/anki/

https://tatoeba.org/en/

https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

#### Cleaining the data

In [29]:
# Clean file
with open("spa.txt", encoding="utf8") as f:
    text = f.read().splitlines()

new = [line.split('CC-BY', 1)[0].strip() for line in text]

In [30]:
# Write cleanded data to tct file
with open('spa-eng.txt', 'w', encoding='utf8') as f:
    for line in new:
        f.write(line)
        f.write('\n')

In [31]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [32]:
EOS = 1
SOS = 0

class wordFunctions:
    def __init__(self, name):
        self.name = name
        self.index2word = {0: "SOS", 1: "EOS"}
        self.word2index = {}
        self.word2count = {}
        self.num_words = 2
    
    def Sentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self,word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1           

In [33]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

In [34]:
# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [35]:
def read(len1, len2, reverse=False):
    # Read lines of processed file and split 
    lines = open('spa-eng.txt', encoding='utf8').read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs and make instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = wordFunctions(len2)
        output_lang = wordFunctions(len1)
    else:
        input_lang = wordFunctions(len1)
        output_lang = wordFunctions(len2)

    return input_lang, output_lang, pairs

In [36]:
def finalData(len1, len2, reverse=False):
    input, output, pairs = read(len1, len2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input.Sentence(pair[0])
        output.Sentence(pair[1])
    print("Counted words:")
    print(input.name, input.num_words)
    print(output.name, output.num_words)
    return input, output, pairs

In [37]:
input_lang, output_lang, pairs = finalData('eng', 'spa', True)

Read 139013 sentence pairs
Trimmed to 139013 sentence pairs
Counting words...
Counted words:
spa 26634
eng 13705


In [38]:

print(random.choice(pairs))

['estoy empezando a perder la paciencia con tom .', 'i m beginning to lose patience with tom .']


In [39]:
MAX_LENGTH = 10

In [40]:
print(len(pairs))

139013


In [41]:
# rnd_idx = 
print(pairs[11:20])

[[' quien ?', 'who ?'], [' orale !', 'wow !'], [' inclinate !', 'duck !'], [' fuego !', 'fire !'], [' incendio !', 'fire !'], [' disparad !', 'fire !'], [' ayuda !', 'help !'], [' socorro ! auxilio !', 'help !'], [' auxilio !', 'help !']]


In [42]:
input_lang.word2index

{'ve': 2,
 '.': 3,
 'vete': 4,
 'vaya': 5,
 'vayase': 6,
 'hola': 7,
 '': 8,
 'corre': 9,
 '!': 10,
 'corran': 11,
 'corra': 12,
 'corred': 13,
 'quien': 14,
 '?': 15,
 'orale': 16,
 'inclinate': 17,
 'fuego': 18,
 'incendio': 19,
 'disparad': 20,
 'ayuda': 21,
 'socorro': 22,
 'auxilio': 23,
 'escondete': 24,
 'salta': 25,
 'salte': 26,
 'salto': 27,
 'quedate': 28,
 'parad': 29,
 'para': 30,
 'pare': 31,
 'espera': 32,
 'esperate': 33,
 'esperen': 34,
 'empieza': 35,
 'continua': 36,
 'continue': 37,
 'date': 38,
 'prisa': 39,
 'daos': 40,
 'dese': 41,
 'me': 42,
 'oculte': 43,
 'escondi': 44,
 'ocultaba': 45,
 'escondia': 46,
 'corri': 47,
 'corria': 48,
 'lo': 49,
 'intento': 50,
 'he': 51,
 'ganado': 52,
 'oh': 53,
 'no': 54,
 'tomatelo': 55,
 'con': 56,
 'soda': 57,
 'disparen': 58,
 'dispara': 59,
 'dispare': 60,
 'sonrie': 61,
 'disculpa': 62,
 'al': 63,
 'ataque': 64,
 'atacad': 65,
 'ataquen': 66,
 'ataca': 67,
 'comprela': 68,
 'cometelo': 69,
 'comeoslo': 70,
 'comaselo': 7

### Let us creat the GRU model

In [43]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, x, hidden):
        embedded = self.embedding(x).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [44]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden):
        output = self.embedding(x).view(1, 1, -1)
#         output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        # print(output.shape)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [45]:
class AttnDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, max_length=MAX_LENGTH):
        super().__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
#         self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
#         self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, x, hidden, encoder_outputs):
        embedded = self.embedding(x).view(1, 1, -1)
#         embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [46]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [47]:
teacher_forcing_ratio = 0.5


def train_once(input_tensor, target_tensor, encoder, decoder,
          encoder_optimizer, decoder_optimizer, criterion, 
          max_length=MAX_LENGTH, attention=True):
    encoder_hidden = encoder.init_hidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):

            if attention:
              decoder_output, decoder_hidden, decoder_attention = decoder(
                  decoder_input, decoder_hidden, encoder_outputs)
            else:
              decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)


            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):

          if attention:
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
          else:
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)


          topv, topi = decoder_output.topk(1)
          decoder_input = topi.squeeze().detach()  # detach from history as input

          loss += criterion(decoder_output, target_tensor[di])
          if decoder_input.item() == EOS:
              break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [48]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [49]:
random.choice(pairs)

['el debe haber perdido el tren .', 'he must have missed the train .']

In [50]:
def train(encoder, decoder, n_iters, print_every=1000, 
               plot_every=100, learning_rate=0.01, attention=True):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train_once(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, 
                     criterion, attention=attention)
        
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [51]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [52]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH, attention=True):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.init_hidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
          if attention:
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
          else:
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)


          if topi.item() == EOS:
              decoded_words.append('<EOS>')
              break
          else:
              decoded_words.append(output_lang.index2word[topi.item()])

          decoder_input = topi.squeeze().detach()

        if attention:
          return decoded_words, decoder_attentions[:di + 1]
        else:
          return decoded_words

In [53]:
def evaluateRandomly(encoder, decoder, n=10, attention=True):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        if attention:
          output_words, attentions = evaluate(encoder, decoder, pair[0], attention=attention)
        else:
          output_words = evaluate(encoder, decoder, pair[0], attention=False)

        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [54]:
hidden_size = 256
encoder1 = Encoder(input_lang.num_words, hidden_size).to(device)
decoder1 = AttnDecoder(hidden_size, output_lang.num_words).to(device)
#decoder1 = Decoder(hidden_size, output_lang.num_words).to(device)
train(encoder1, decoder1, 50000, print_every=5000, attention=True)

IndexError: index 10 is out of bounds for dimension 0 with size 10

In [ ]:
evaluateRandomly(encoder1, decoder1, attention=True)